### Rectify all images on S3 bucket and write to public read bucket

In [1]:
from pathlib import Path
import imageio
import fsspec
import numpy as np
import matplotlib.pyplot as plt
import datetime
from dateutil import tz
import pandas as pd

from coastcam_funcs import *
from calibration_crs import *
from rectifier_crs import *

#### Read the inventory of products  
Generated with `aws s3 ls s3://cmgp-coastcam/cameras/caco-01/products/ --profile coastcam > caco01inventory.txt`  
Then edited to remove a few bad entries.
These have beeen deleted from the S3 bucket.

In [2]:
df = pd.read_csv('caco01inventory.txt', header=None, delim_whitespace=True, parse_dates={'datetime': [0, 1]})
df.rename(columns={2:'fid',3:'filename'}, inplace=True)
del df['fid']
df

,datetime,filename
0,2019-12-13 18:45:15,1576260000.c1.snap.jpg
1,2019-12-13 18:45:15,1576260000.c2.snap.jpg
2,2019-12-13 18:45:16,1576260001.c1.timex.jpg
3,2019-12-13 18:45:17,1576260001.c2.timex.jpg
4,2019-12-13 18:45:18,1576260002.c1.var.jpg
...,...,...
103462,2020-11-12 21:01:30,1605213003.c2.bright.jpg
103463,2020-11-12 21:01:31,1605213004.c1.dark.jpg
103464,2020-11-12 21:01:33,1605213004.c2.dark.jpg
103465,2020-11-12 21:01:35,1605213005.c1.rundark.jpg


### Get list of `c1.timex` files and find matching `c2.timex` files. Make list of `c1.timex` images with matches.

In [3]:
# find instances of timex.c1
df2 = df[df['filename'].str.contains('c1.timex')]
df2

,datetime,filename
2,2019-12-13 18:45:16,1576260001.c1.timex.jpg
14,2019-12-13 18:45:24,1576261801.c1.timex.jpg
26,2019-12-13 19:45:09,1576263601.c1.timex.jpg
38,2019-12-13 19:45:15,1576265401.c1.timex.jpg
50,2019-12-13 20:45:09,1576267201.c1.timex.jpg
...,...,...
103409,2020-11-12 19:04:42,1605205801.c1.timex.jpg
103421,2020-11-12 20:00:54,1605207601.c1.timex.jpg
103433,2020-11-12 20:01:14,1605209401.c1.timex.jpg
103445,2020-11-12 21:01:01,1605211201.c1.timex.jpg


In [4]:
# filelist will contain all of the c1.timex images that have matching c2 images
filelist = [ ]
matches = 0
no_matches = 0
for i, row in df2.iterrows():
    fs1 = row['filename']
    # print(fs1)
    fn1 = list(fs1)
    fn2 = fn1.copy()
    fn2[12]='2'
    fs2="".join(fn2)
    # print(fs2)
    if len(df[df['filename'].isin([fs2])]) == 1:
        filelist.append(fs1)
        matches += 1
    else:
        no_matches += 1

print(matches, no_matches, matches+no_matches)

8231 761 8992


### Set up the rectification

In [5]:
# List of files...three for each camera. Calibration parameters are in .json format
# These are the USGS image filename format
extrinsic_cal_files = ['CACO01_C1_EOBest.json','CACO01_C2_EOBest.json']
intrinsic_cal_files = ['CACO01_C1_IOBest.json','CACO01_C2_IOBest.json']

# Dict providing the metadata that the Axiom code infers from the USACE filename format
metadata= {'name': 'CACO-01', 'serial_number': 1, 'camera_number': 'C1', 'calibration_date': '2019-12-12', 'coordinate_system': 'geo'}
# dict providing origin and orientation of the local grid
local_origin = {'x': 410935.,'y':4655890., 'angd': 55.}

# read cal files and make lists of cal dicts
extrinsics_list = []
for f in extrinsic_cal_files:
    extrinsics_list.append( json2dict(f) )
intrinsics_list = []
for f in intrinsic_cal_files:
    intrinsics_list.append( json2dict(f) )
    
# check test for coordinate system
if metadata['coordinate_system'].lower() == 'xyz':
    print('Extrinsics are local coordinates')
elif metadata['coordinate_system'].lower() == 'geo':
    print('Extrinsics are in world coordinates')
else:
    print('Invalid value of coordinate_system: ',metadata['coordinate_system'])
    
print(extrinsics_list[0])
print(extrinsics_list[0]['y']-local_origin['y'])

calibration = CameraCalibration(metadata,intrinsics_list[0],extrinsics_list[0],local_origin)
print(calibration.local_origin)
print(calibration.world_extrinsics)
print(calibration.local_extrinsics)

xmin = 0.
xmax = 500.
ymin = 0.
ymax = 700.
dx = 1.
dy = 1.
z =  0.

rectifier_grid = TargetGrid(
    [xmin, xmax],
    [ymin, ymax],
    dx,
    dy,
    z
)
#print(rectifier_grid.X)

rectifier = Rectifier(
    rectifier_grid
)

Extrinsics are in world coordinates
{'x': 410843.97, 'y': 4655942.49, 'z': 27.3, 'a': -0.271, 't': 1.304, 'r': 0.007}
52.49000000022352
{'x': 410935.0, 'y': 4655890.0, 'angd': 55.0}
{'x': 410843.97, 'y': 4655942.49, 'z': 27.3, 'a': -0.271, 't': 1.304, 'r': 0.007}
{'x': -9.215372196139484, 'y': 104.67443773584442, 'z': 27.3, 'a': 0.6889310885968812, 't': 1.304, 'r': 0.007}


In [6]:
# setup S3 buckets
imdir='cmgp-coastcam/cameras/caco-01/products/'
fs = fsspec.filesystem('s3',profile='coastcam')

fs2 = fsspec.filesystem('s3', profile='default')


In [7]:
# function to create a c2 image name from a c1 image name
def c1toc2(c1name):
    fn1 = list(c1name)
    fn2 = fn1.copy()
    fn2[12]='2'
    fs2="".join(fn2)
    return fs2

# test it
print(filelist[22])
print(c1toc2(filelist[22]))

1576445401.c1.timex.jpg
1576445401.c2.timex.jpg


In [12]:
range(6150,len(filelist))[-1]

8230

In [16]:
# loop through the filelist and calculate image time, brightness, sharpness, and contrast.
# If the brightness > 40, rectify the image

# Some switches because I was developing this incrementally
do_rectify = True
jpeg_to_bucket = False
png_to_bucket = True
stats_to_csv = True
#if stats_to_csv:
    # set up file for output
#     with open('caco_rectify_stats.csv','w') as csvfile:
#         csvfile.write('filname, time (UTC),avg0,s0,c0,blur0,avg1,s1,c1,blur1'+'\n')

icount = 0
#for i, fn in enumerate(filelist):
for i in range(6150,len(filelist)):
    fn = filelist[i]
    fpath = imdir+fn
    
    #output filename for rectified jpg
    ofile = fn.split('.')[0]+'.jpg'
    
    # second path is same but for camera 2
    impaths = [fpath, imdir+c1toc2(fn)]

    s0, c0 = estimate_sharpness(impaths[0],fs)
    rgb0, avg0 = average_color(impaths[0],fs)
    blur0 = detect_blur_fft(impaths[0],vis=False,fs=fs)
    s1, c1 = estimate_sharpness(impaths[1],fs)
    rgb1, avg1 = average_color(impaths[1],fs)
    blur1 = detect_blur_fft(impaths[1],vis=False,fs=fs)    
    
    ftime, e = filetime2timestr(fn, timezone='eastern')
#     print(ofile)
#     print(impaths)
#     print(i,fn,ftime,avg,s)
    if avg0 > 40. :
        icount += 1
        if do_rectify:
            rectified_image = rectifier.rectify_images(metadata, impaths, intrinsics_list, \
                                                            extrinsics_list, local_origin, fs=fs)        
        if jpeg_to_bucket:
            # write to the public sfm bucket
            with fs2.open('s3://cmgp-sfm-public-read-bucket/csherwood/caco01_rectified/'+ofile,'wb') as fo:
                imageio.imwrite(fo,np.flip(rectified_image,0),format='jpg')
                
        if png_to_bucket:
            # make an annotated image
            fig=plt.imshow( rectified_image.astype(np.int))
            plt.gca().invert_yaxis()
            plt.xlabel('Offshore (m)')
            plt.ylabel('Alongshore (m)')
            # make a North arrow
            angr = calibration.local_origin['angd']*np.pi/180.
            dx = np.cos(angr)*90.
            dy = np.sin(angr)*90.
            plt.arrow(50,550,dx,dy,linewidth=2,head_width=25,head_length=30,color='white',shape='right')
            plt.text(100,670,'N',color='white')
            plt.text(220,670,ftime,fontsize=8,color='gold')
            ts = ('B={:.0f} S={:.1f} C={:.0f}'.format(avg0,s0,c0))
            plt.text(220,10,ts,fontsize=8,color='gold')
            plt.xticks([0,100,200,300,400,500],['0','100','200','300','400','500'])
            plt.title(e)
            fp = e+'.png'
            with fs2.open('s3://cmgp-sfm-public-read-bucket/csherwood/caco01_rectified_pngs/'+fp,'wb') as fopng:
                plt.savefig(fopng,dpi=200)
            #plt.show()
            plt.close()
        
        if stats_to_csv:
            ostring = '{}.jpg,{},{:.0f},{:3.1f},{:5.1f},{:5.1f},{:.0f},{:3.1f},{:5.1f},{:5.1f}'.format(e,ftime,avg0,s0,c0,blur0,avg1,s1,c1,blur1)
            #print(ostring)
            with open('caco_rectify_stats.csv', 'a') as csvfile:
                csvfile.write(ostring+'\n')
            
        if not(icount % 20):
            print(icount)
        
#         if icount > 11:
#             break
            
print(icount,' images processed.')                    

20
40
60
80
100
120
140
160
180
200
220
240
260
280
300
320
340
360
380
400
420
440
460
480
500
520
540
560
580
600
620
640
660
680
700
720
740
760
780
800
820
840
860
880
900
920
940
960
980
1000
1020
1040
1060
1080
1100
1120
1140
1160
1180
1200
1220
1240
1260
1280
1300
1320
1340
1360
1380
1400
1420
1440
1460
1480
1500
1520
1540
1560
1580
1600
1620
1640
1660
1680
1700
1720
1740
1760
1780
1800
1810  images processed.
